## Example of Computing Implicit Prices from Consumption & Expenditures



In [1]:
import cfe
import pandas as pd
import numpy as np

Sheets = {"ICRISAT_wprices":"13Ig5hZif-NSHtgkKRp_cEgKXk0lOsdUB2BAD6O_FnRo"}

def dataframe_from_gspreadsheet(sheet_name, key):
    """Transform public google spreadsheet into pandas.DataFrame."""
    
    url = 'https://docs.google.com/spreadsheets/d/{key}/gviz/tq?tqx=out:csv&sheet={sheet_name}&headers=1'.format(
        key=key, sheet_name=sheet_name.replace(' ', '%20'))

    print(url)
    df = pd.read_csv(url,dtype={'j':str})

    df.columns = [c.strip() for c in df.columns.tolist()]

    df = df.loc[:,~df.columns.duplicated(keep='first')]   

    df = df.drop([col for col in df.columns if col.startswith('Unnamed')], axis=1)

    df = df.loc[~df.index.duplicated(), :]

    return df

def prices(Q,X,tol=1e-6):
    """Impute prices from data on expenditures and quantities.

    Non-trivial because quantities may be reported in different units.
    """
    
    myQ = Q.groupby(['j','t','m','unit']).sum()

    B={}
    for t in myQ.index.levels[1]:
        for m in myQ.index.levels[2]:
            for i in myQ.columns:
                try:
                    useX = X.query("t==%d and m=='%s'" % (t,m))[i].fillna(0)
                    useQ = myQ.query("t==%d and m=='%s'" % (t,m))[i].fillna(0).unstack('unit')
                    if len(useX):
                        q,x = useQ.fillna(0).align(useX.fillna(0),axis=0,join='inner')
                        b = np.linalg.lstsq(q,x,rcond=None)[0]
                        b = pd.Series(b,index=q.columns,name=i)
                        B[(t,m,i)] = b.where(b>0,0).round(6)
                except KeyError:  # i not in X?
                    pass

    P = pd.concat(B).replace(0,np.nan).dropna()
    P.index.names = ['t','m','i','unit']

    P = P.unstack(['i','unit'])
    
    return P

for k,v in Sheets.items():
    print(k)
    x = dataframe_from_gspreadsheet("Expenditures", Sheets[k])
    c = dataframe_from_gspreadsheet("Consumption", Sheets[k])

    z = dataframe_from_gspreadsheet("Household Characteristics", Sheets[k])

    # If no 'm' index assume a single market:
    if 'm' not in z.index.names and 'm' not in z.columns:
        z['m'] = 1
        x['m'] = 1
        c['m'] = 1

    z = z.reset_index().set_index(['j','t','m'])
    c = c.reset_index().set_index(['j','t','m'])
    x = x.reset_index().set_index(['j','t','m'])

    x = x.loc[~x.index.duplicated(), :]
    c = c.loc[~c.index.duplicated(), :]
    z = z.loc[~z.index.duplicated(), :]

    p = prices(c,x)  # Prices for all transactions

    # Median price for different goods in different units
    # in different markets and in different years 
    p.groupby(['m','t']).median()
    

/home/ethanligon/anaconda3/lib/python3.7/typing.py:845: FutureWarning: xarray subclass Result should explicitly define __slots__
  super().__init_subclass__(*args, **kwargs)


ICRISAT_wprices
https://docs.google.com/spreadsheets/d/13Ig5hZif-NSHtgkKRp_cEgKXk0lOsdUB2BAD6O_FnRo/gviz/tq?tqx=out:csv&sheet=Expenditures&headers=1
https://docs.google.com/spreadsheets/d/13Ig5hZif-NSHtgkKRp_cEgKXk0lOsdUB2BAD6O_FnRo/gviz/tq?tqx=out:csv&sheet=Consumption&headers=1
https://docs.google.com/spreadsheets/d/13Ig5hZif-NSHtgkKRp_cEgKXk0lOsdUB2BAD6O_FnRo/gviz/tq?tqx=out:csv&sheet=Household%20Characteristics&headers=1


In [2]:
p.groupby(['m','t']).median()

i        index Other Food Edible oils       Fish       Ghee       Milk  \
unit Kilograms  Kilograms   Kilograms  Kilograms  Kilograms  Kilograms   
m t                                                                      
A 75  0.407239   0.800000    7.590522   4.000000  14.748448   7.900000   
  76  0.376007   0.958811    5.624942   4.000000  14.521495        NaN   
  77  0.358498   2.100000         NaN   3.406829        NaN        NaN   
  78  0.351745        NaN         NaN   4.000000        NaN        NaN   
  79  0.347601        NaN         NaN   5.548459  16.000000        NaN   
  80  0.344814        NaN         NaN   4.301619  16.000000        NaN   
  81  0.342235        NaN   13.892649   5.239339  19.186289        NaN   
  82  0.340118        NaN         NaN   8.872592  20.018433        NaN   
  83  0.348736        NaN         NaN        NaN  24.000000        NaN   
  84  0.362882        NaN         NaN        NaN        NaN        NaN   
C 75  0.365424   6.000000   10.250769   4.500000  20.000000        NaN   
  76  0.362807   1.102943    6.746957   4.589744  21.816653        NaN   
  77  0.359662        NaN   10.346756   4.581688  24.596273        NaN   
  78  0.357382  16.000000   11.344597   6.345353  21.050955        NaN   
  79  0.355522        NaN   10.341654   5.138589  20.000000        NaN   
  80  0.353697        NaN   10.729725   7.168332  36.994709  23.976364   
  81  0.351628        NaN   13.166483   6.926174  30.132713        NaN   
  82  0.350108   2.000000   13.023137   9.600000  29.857143        NaN   
  83  0.351977        NaN         NaN        NaN        NaN        NaN   
  84  0.356531        NaN         NaN        NaN        NaN        NaN   
D 77  0.359075        NaN         NaN        NaN        NaN        NaN   
E 75  0.358115        NaN    6.712392   4.159364  19.013384        NaN   
  76  0.357210        NaN    6.159375   4.549112  18.488031        NaN   
  77  0.356110   0.400000    8.992979   5.271584  21.974770        NaN   
  78  0.355039   3.100000    8.135333   6.007417  24.152254        NaN   
  79  0.354078        NaN    9.996838   6.326087  28.375480        NaN   
  80  0.353190        NaN   11.534826   6.827789  13.629607        NaN   
  81  0.352527        NaN   15.064724  10.285968  28.844504        NaN   
  82  0.351701        NaN   14.410172  10.206327  32.769625        NaN   
  83  0.353417        NaN   14.799490        NaN  40.000000        NaN   
  84  0.357769        NaN         NaN        NaN        NaN        NaN   
F 77  0.360188        NaN         NaN        NaN        NaN        NaN   

i       Castor Groundnut oil Castor (HYV) Safflower oil  ...  \
unit Kilograms     Kilograms    Kilograms     Kilograms  ...   
m t                                                      ...   
A 75  1.510760      8.000000     3.356959      9.488009  ...   
  76  1.361848      7.086324          NaN           NaN  ...   
  77  2.110547      8.194463          NaN           NaN  ...   
  78  0.746699      7.633370          NaN           NaN  ...   
  79  1.722703      9.435854          NaN           NaN  ...   
  80  2.455741     10.582977          NaN      8.682448  ...   
  81  2.845855     13.880995     2.700000           NaN  ...   
  82  2.782424     13.900857     2.791009     13.575710  ...   
  83  3.130570           NaN     4.220665     12.051804  ...   
  84  4.000000           NaN     5.194164      8.333333  ...   
C 75       NaN           NaN          NaN           NaN  ...   
  76       NaN           NaN          NaN           NaN  ...   
  77       NaN      9.126270          NaN           NaN  ...   
  78       NaN      7.866957          NaN           NaN  ...   
  79       NaN      8.359076          NaN           NaN  ...   
  80       NaN     11.063248          NaN           NaN  ...   
  81       NaN     12.340411          NaN           NaN  ...   
  82       NaN           NaN          NaN           NaN  ...   
  83       NaN           NaN          NaN     17.230769  ...   
 